<a href="https://colab.research.google.com/github/bkeuthan/Spark2/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (212 kB/s)
Reading package lists... Done


In [42]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [43]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [44]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show()
df.createOrReplaceTempView("temp_view")


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [45]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS AveragePrice FROM temp_view_name WHERE bedrooms = 4 GROUP BY Year ORDER BY Year")
result.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [46]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS AveragePrice FROM temp_view_name WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
result.show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   292859.62|
|      2011|   291117.47|
|      2012|   293683.19|
|      2013|   295962.27|
|      2014|   290852.27|
|      2015|    288770.3|
|      2016|   290555.07|
|      2017|   292676.79|
+----------+------------+



In [47]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS AveragePrice FROM temp_view_name WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
result.show()

+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   285010.22|
|      2011|   276553.81|
|      2012|   307539.97|
|      2013|   303676.79|
|      2014|   298264.72|
|      2015|   297609.97|
|      2016|    293965.1|
|      2017|   280317.58|
+----------+------------+



In [48]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

--- 8.749961853027344e-05 seconds ---


In [49]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("temp_view")

In [50]:
# 8. Check if the table is cached.
spark.catalog.isCached('temp_view')

True

In [51]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 7.319450378417969e-05 seconds ---


In [55]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import shutil

# Specify the output directory
output_directory = "partitioned_home_sales.parquet"

# Delete the existing directory if it exists
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

# Write the partitioned data to the specified directory
df.write.partitionBy("date_built").parquet(output_directory)

In [56]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("partitioned_home_sales.parquet")
parquet_df.show()

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a

In [57]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data_temp_view")

In [58]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 5.364418029785156e-05 seconds ---


In [59]:
# Uncache the home_sales temporary table
spark.catalog.uncacheTable("parquet_data_temp_view")


In [60]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('parquet_data_temp_view')

False